In [79]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# from unsloth import FastLanguageModel
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import init_chat_model
from langchain import hub
from typing_extensions import List, TypedDict, Optional
from langgraph.graph import START, StateGraph
from langchain_core.documents import Document

import glob

import pandas as pd

In [80]:
with open('api_key.txt', 'r') as file:
    API_KEY = file.read().strip()

# LOAD VECTOR DATA

In [84]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

vectorstore = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

# Fireworks AI

In [85]:
llm = init_chat_model(
    "accounts/fireworks/models/deepseek-v3",
    # "accounts/fireworks/models/qwen3-30b-a3b-instruct-2507", 
    # "accounts/fireworks/models/gpt-oss-20b",
    # "accounts/fireworks/models/gpt-oss-120b",
    model_provider="fireworks", 
    api_key=API_KEY
)

# MODEL = "gpt-oss-20b"
# MODEL = "gpt-oss-120b"
# MODEL = "qwen3-30b"
MODEL = "deepseek-v3"


In [86]:
prompt = hub.pull("rlm/rag-prompt")

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str
    
    # generation & retrieval controls
    max_tokens: Optional[int]
    top_p: Optional[float]
    top_k: Optional[int]
    presence_penalty: Optional[float]
    frequency_penalty: Optional[float]
    temperature: Optional[float]

def retrieve(state: State):
    retrieved_docs = vectorstore.similarity_search(state["question"])
    
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)

    return {"answer": response.content}

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [87]:
instruction = "You are an empathic chatbot designed to support young caregivers and young carers. You will be given a question by a patient. Answer the following question with help from the given context: "

# generate answers

In [88]:
df = pd.read_csv('testset.csv')
df['Answer'] = None
df

,Context,Response,Answer
0,We weren't long distance until he joined the m...,You're wise to be aware of possible changes to...,None
1,I have no real friends. I have a girlfriend wh...,Having time all to yourself may be necessary s...,None
2,I feel like my time is going too fast,Thank you for reaching out! That is a great qu...,None
3,He wants to wear makeup and heels. He even tuc...,It sounds like you may be asking two different...,None
4,I am a really shy person. I'm currently in a g...,"First of all, way to go in stretching yourself...",None
...,...,...,...
195,"Maybe this is a stupid question, but I sometim...","It is not a stupid question, it is very basi...",None
196,I feel that I need to end my present relations...,It sounds like you have some insight into the ...,None
197,How do I decide if I should end my relationshi...,"That's a loaded question. Typically, if we ar...",None
198,I don't know what to say. I have never really ...,"The older I get, the more I believe that our r...",None


In [89]:
for i, row in df.iterrows():
    question = instruction + row['Context']
    # response = graph.invoke({"question": question})
    response = graph.invoke({
        "question": question,
        "max_tokens": 4096,
        "top_p": 1,
        "top_k": 40,
        "presence_penalty": 0,
        "frequency_penalty": 0,
        "temperature": 0.6,
    })
    df.at[i, 'Answer'] = response['answer']

    if i % 5 == 0:
        print(f"Processed row {i+1}/{len(df)}")
        print(response['answer'])

Processed row 1/200
Maintaining a long-distance relationship while your partner is in the military can be challenging, but open communication and trust are key. Make an effort to stay connected through regular check-ins, shared activities, or even small gestures like letters or care packages. Embrace your strengths and work together to navigate this phase, knowing that your love and commitment can help you both through it.
Processed row 6/200
Quitting smoking is challenging, especially when it’s tied to mental habits rather than just physical cravings. To support your journey, consider speaking with a counsellor or local mental health nurse for emotional guidance and strategies to manage the mental aspect of addiction. Additionally, connecting with supportive friends, family, or a trusted health professional can provide encouragement and accountability. Remember, taking care of yourself is crucial for both your well-being and your future plans.
Processed row 11/200
To fight winter sadn

In [90]:
df.to_csv(f'{MODEL}_RAG_baseline1.csv', index=False)

In [91]:
df

,Context,Response,Answer
0,We weren't long distance until he joined the m...,You're wise to be aware of possible changes to...,Maintaining a long-distance relationship while...
1,I have no real friends. I have a girlfriend wh...,Having time all to yourself may be necessary s...,I’m sorry you’re feeling so alone right now. T...
2,I feel like my time is going too fast,Thank you for reaching out! That is a great qu...,Feeling like time is going too fast can be ove...
3,He wants to wear makeup and heels. He even tuc...,It sounds like you may be asking two different...,It’s important to prioritize your comfort and ...
4,I am a really shy person. I'm currently in a g...,"First of all, way to go in stretching yourself...","To build confidence in presentations, start by..."
...,...,...,...
195,"Maybe this is a stupid question, but I sometim...","It is not a stupid question, it is very basi...","It’s not a stupid question, and feeling unsure..."
196,I feel that I need to end my present relations...,It sounds like you have some insight into the ...,It sounds like you’re struggling with a diffic...
197,How do I decide if I should end my relationshi...,"That's a loaded question. Typically, if we ar...",Deciding whether to end a relationship is a de...
198,I don't know what to say. I have never really ...,"The older I get, the more I believe that our r...","It’s okay to feel unsure about who you are, es..."
